In [66]:
#Imported relevant and necessary libraries and data cleaning tools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import hypertools as hyp
from glob import glob as lsdir
import os
import re
import datetime as dt

from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

%matplotlib inline

In [67]:
#Code from Professor Manning to set up and read in the relevant UVLT data 
data_readers = {'xlsx': pd.read_excel, 'xls': pd.read_excel, 'dta': pd.read_stata}
get_extension = lambda x: x.split('.')[-1]


def read_data(datadir, readers):
    files = lsdir(os.path.join(datadir, '*'))
    readable_files = []
    data = []
    for f in files:
        ext = get_extension(f)
        if ext in readers.keys():
            readable_files.append(f)
            data.append(data_readers[ext](f))
    return readable_files, data


fnames, data = read_data('data', data_readers)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [68]:
#A summary of the data files that are now read into the notebook
fnames

['data/UVLTdata_individual.dta',
 'data/UVLTdata_final.dta',
 'data/Direct Mailing Analysis.xlsx',
 'data/UVLTDataAnalysis.xls',
 'data/TownLevelData.xlsx',
 'data/TownLevelData.dta',
 'data/CensusInfoUpperValley2015JH.xlsx']

In [69]:
#This is the individual data
#Most of the initial cleaning that I have done has been on this data set, though obviously it can be applied if necessary to any of the data sets
data[0].head()

,ContactID,State,TownID,Town,LandOwnerTownID,DeceasedDateYN,U_Tot_Amt,U_Tot_Cnt,U200001,U200102,...,E201112,E201213,E201314,E201415,E201516,E201617,E201718,E201819,DeceasedDate,ConservedOwner
0,1544.0,NC,60.0,All Other Towns,0.0,0.0,571.95,6.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1,1545.0,NH,20.0,Lebanon,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2,1546.0,NH,61.0,All Other Towns NH,0.0,0.0,600.00,7.0,25.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
3,1547.0,VT,41.0,Weathersfield,41.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
4,1548.0,VT,34.0,Springfield,0.0,1.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016.0,0.0


In [99]:
#Renaming relevant columns in UVLT individual data to be more easily readable
names={'DeceasedDateYN' : 'Is the donor Deceased?',
       'U_Tot_Amt': 'Total Unrestricted Donations',
      'U_Tot_Cnt': 'Total Number of Unrestricted Donations Given',
      'ConservedOwner' : 'Owns Conserved Land?',
      'RTotAmt' : 'Total Restricted Donations',
       'RTotCnt': 'Total Number of Restricted Donations Given',
      'VTotCnt' : 'Total Volunteer Occurances',
      'ETotCnt' : 'Total Event Attendances'}
data[0].rename(names, inplace=True, axis=1)

In [110]:
#Summary of the column values in data set 1
data[0].columns.values

array(['ContactID', 'State', 'TownID', 'Town', 'LandOwnerTownID',
       'Is the donor Deceased?', 'Total Unrestricted Donations',
       'Total Number of Unrestricted Donations Given', 'U200001',
       'U200102', 'U200203', 'U200304', 'U200405', 'U200506', 'U200607',
       'U200708', 'U200809', 'U200910', 'U201011', 'U201112', 'U201213',
       'U201314', 'U201415', 'U201516', 'U201617', 'U201718', 'U201819',
       'Total Restricted Donations',
       'Total Number of Restricted Donations Given', 'R200001', 'R200102',
       'R200203', 'R200304', 'R200405', 'R200506', 'R200607', 'R200708',
       'R200809', 'R200910', 'R201011', 'R201112', 'R201213', 'R201314',
       'R201415', 'R201516', 'R201617', 'R201718', 'R201819',
       'Total Volunteer Occurances', 'V200001', 'V200102', 'V200203',
       'V200304', 'V200405', 'V200506', 'V200607', 'V200708', 'V200809',
       'V200910', 'V201011', 'V201112', 'V201213', 'V201314', 'V201415',
       'V201516', 'V201617', 'V201718', 'V201819

In [100]:
#copying each set of data into more memorably named versions
#I figured different analyses require different aspects of each dataframe, so starting here and using copies of the data for different analyses may be helpful for organizationa and fidelity
Individual_data=data[0].copy()
Final_data=data[1].copy()
Mailing_data=data[2].copy()
Town_Data=data[5].copy()


#Similarly to the individual data, the final data could benefit from some cleaner column names
Final_data.rename(names, inplace=True, axis=1)

In [105]:
to_drop={'U200001',
        'U200102',
        'U200203',
        'U200304',
        'U200405',
        'U200506',
        'U200607',
        'U200708',
        'U200809',
        'U200910',
        'U201011',
        'U201112',
        'U201213',
        'U201314',
        'U201415',
        'U201516',
        'U201617',
        'U201718',
        'U201819',
        'R200001', 
        'R200102',
        'R200203', 
        'R200304',
        'R200405', 
        'R200506', 
        'R200607', 
        'R200708', 
        'R200809', 
        'R200910',
        'R201011', 
        'R201112', 
        'R201213', 
        'R201314', 
        'R201415', 
        'R201516',
        'R201617', 
        'R201718', 
        'R201819',
        'V200001', 
        'V200102',
        'V200203', 
        'V200304', 
        'V200405', 
        'V200506', 
        'V200607', 
        'V200708',
        'V200809', 
        'V200910', 
        'V201011', 
        'V201112', 
        'V201213', 
        'V201314',
        'V201415', 
        'V201516', 
        'V201617', 
        'V201718', 
        'V201819',
        'E200001', 
        'E200102', 
        'E200203', 
        'E200304', 
        'E200405', 
        'E200506',
        'E200607', 
        'E200708', 
        'E200809',
        'E200910', 
        'E201011', 
        'E201112',
        'E201213',
        'E201314', 
        'E201415', 
        'E201516', 
        'E201617', 
        'E201718',
        'E201819'}

Individual_data_SUMMARY=data[0].copy()

In [106]:
Individual_data_SUMMARY.drop(to_drop, inplace=True, axis=1)

In [107]:
#The summary cleaning gets rid of the year-by-year data and leaves only the summary of the donations for analyses
#obviously there are pros and cons to this, but I figured it was a tedious process to undertake so now it's a simple dictionary that can be used on the various data sets
Individual_data_SUMMARY.head(15)

,ContactID,State,TownID,Town,LandOwnerTownID,Is the donor Deceased?,Total Unrestricted Donations,Total Number of Unrestricted Donations Given,Total Restricted Donations,Total Number of Restricted Donations Given,Total Volunteer Occurances,Total Event Attendances,DeceasedDate,Owns Conserved Land?
0,1544.0,NC,60.0,All Other Towns,0.0,0.0,571.95,6.0,0.0,0.0,0.0,0.0,NaN,0.0
1,1545.0,NH,20.0,Lebanon,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2,1546.0,NH,61.0,All Other Towns NH,0.0,0.0,600.00,7.0,0.0,0.0,0.0,0.0,NaN,0.0
3,1547.0,VT,41.0,Weathersfield,41.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,NaN,1.0
4,1548.0,VT,34.0,Springfield,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,2016.0,0.0
5,1549.0,VT,13.0,Fairlee,13.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,NaN,1.0
6,1550.0,NH,28.0,Plainfield,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,NaN,0.0
7,1551.0,VT,36.0,Thetford,36.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8,1552.0,NH,20.0,Lebanon,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,2014.0,0.0
9,1553.0,VT,40.0,Vershire,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,2014.0,0.0


In [108]:
Final_data_SUMMARY=Final_data.copy()
Final_data_SUMMARY.drop(to_drop, inplace=True, axis=1)

In [109]:
Final_data_SUMMARY.head(15)

,ContactID,State,TownID,Town,LandOwnerTownID,Is the donor Deceased?,Total Unrestricted Donations,Total Number of Unrestricted Donations Given,Total Restricted Donations,Total Number of Restricted Donations Given,...,Total Event Attendances,DeceasedDate,Owns Conserved Land?,Nprojects,Nacres,Nmembers,MedianHHIncome,MeanHHIncome,PercBAplus,PercAge55Plus
0,2903.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
1,11472.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
2,9206.0,VT,2.0,Bradford,0.0,0.0,75.0,2.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
3,12910.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
4,5029.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
5,13212.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
6,6363.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
7,13203.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
8,11538.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
9,13205.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
